## Using custom NNPs with Auto3D

Auto3D (>= 2.3.0) is compatible with any jitable NNPs. This notebook demonstrates how to wrapper and jit a custom NNP to a specific format that Auto3D can use.

In [2]:
import os, sys
root = os.path.dirname(os.path.dirname(os.path.abspath("__file__")))

import torch
import torchani
import Auto3D
from Auto3D.auto3D import options, main

print(Auto3D.__version__)

2.3.0


In [3]:
# Below is the template of the wrapper that you need to implement with your custom NNP.
class userNNP(torch.nn.Module):
    def __init__(self):
        super(userNNP, self).__init__()
        """This is an example NNP model that can be used with Auto3D.
        You can initialize an NNP model however you want,
        just make sure that:
            - It contains the coord_pad and species_pad attributes 
              (These values will be used when processing the molecules in batch.)
            - The signature of the forward method is the same as below.
        """
        # Here I constructed an example NNP using ANI2x.
        # In your case, you can replace this with your own NNP model.
        self.model = torchani.models.ANI2x(periodic_table_index=True)

        self.coord_pad = 0  # int, the padding value for coordinates
        self.species_pad = -1  # int, the padding value for species.

    def forward(self,
                species: torch.Tensor,
                coords: torch.Tensor,
                charges: torch.Tensor) -> torch.Tensor:
        """
        Your NNP should take species, coords, and charges as input
        and return the energies of the molecules.

        species contains the atomic numbers of the atoms in the molecule: [B, N]
        where B is the batch size, N is the number of atoms in the largest molecule.
        
        coords contains the coordinates of the atoms in the molecule: [B, N, 3]
        where B is the batch size, N is the number of atoms in the largest molecule,
        and 3 represents the x, y, z coordinates.
        
        charges contains the molecular charges: [B]
        
        The forward function returns the energies of the molecules: [B],
        output energy unit: eV"""

        # an example for computing molecular energy, replace with your NNP model
        energies = self.model((species, coords)).energies * 27.211386245988
        return energies

In [5]:
# initialize and jit the wrapper with your NNP model
myNNP = userNNP()
myNNP_jit = torch.jit.script(myNNP)

# save the model to a file for later use
model_path = os.path.join(root, 'myNNP.pt')
torch.jit.save(myNNP_jit, model_path)

/home/jack/miniconda3/envs/py39/lib/python3.9/site-packages/torchani/resources/


In [6]:
# Now you can run Auto3D with your custom NNP model.
# Simply parse the model_path to the optimizing_engine arguement

smi_path = os.path.join(root, "example/files/smiles.smi")  # You can specify the path to your file here
args = options(smi_path, k=1, optimizing_engine=model_path, use_gpu=True, gpu_idx=0)
out = main(args)
print(out)

Checking input file...
	There are 4 SMILES in the input file /home/jack/Auto3D_pkg/example/files/smiles.smi. 
	All SMILES and IDs are valid.
Suggestions for choosing isomer_engine and optimizing_engine: 
	Isomer engine options: RDKit and Omega.
	Optimizing engine options: ANI2x, ANI2xt, AIMNET or your own NNP.
The available memory is 24 GB.
The task will be divided into 1 jobs.
Job1, number of inputs: 4


Isomer generation for job1
Enumerating cis/tran isomers for unspecified double bonds...
Enumerating R/S isomers for unspecified atomic centers...
Removing enantiomers...
Enumerating conformers/rotamers, removing duplicates...


100%|██████████| 4/4 [00:00<00:00, 13.09it/s]




Optimizing on job1
Loading model from /home/jack/Auto3D_pkg/myNNP.pt
Preparing for parallel optimizing... (Max optimization steps: 5000)
Total 3D conformers: 45


 10%|█         | 500/5000 [00:13<02:14, 33.41it/s]

Total 3D structures: 45  Converged: 18   Dropped(Oscillating): 0    Active: 27


...

Total 3D structures: 45  Converged: 41   Dropped(Oscillating): 0    Active: 4


 40%|████      | 2000/5000 [00:47<01:03, 46.89it/s]

Total 3D structures: 45  Converged: 44   Dropped(Oscillating): 0    Active: 1


 43%|████▎     | 2126/5000 [00:49<01:07, 42.59it/s]


Optimization finished at step 2127:   Total 3D structures: 45  Converged: 45   Dropped(Oscillating): 0    Active: 0
Begin to select structures that satisfy the requirements...
Energy unit: Hartree if implicit.
Program running time: 1 minute(s)
Output path: /home/jack/Auto3D_pkg/example/files/smiles_20240802-014545-537526/smiles_out.sdf
/home/jack/Auto3D_pkg/example/files/smiles_20240802-014545-537526/smiles_out.sdf
